# Happy Portfolio Analysis

In [81]:
import pickle
import pandas as pd
import plotly.express as px
import numpy as np


In [87]:
# Read in etf data from pkl file

pkl_file = open('etf_df.pkl', 'rb')
dataframe2 = pickle.load(pkl_file)
pkl_file.close()

# load etf data back into Dataframe
all_etf_df = pd.DataFrame(dataframe2)

# Convert month end prices to monthly percentage change and drop na values

all_etf_df = all_etf_df.pct_change().dropna()


In [8]:
# Read in country data from pkl file

pkl_file = open('wh_2015_2019.pkl', 'rb')
dataframe3 = pickle.load(pkl_file)
pkl_file.close()

# load etf data back into Dataframe
wh_2015_2019_df = pd.DataFrame(dataframe3)

In [9]:
wh_2015_2019_df

Country,Switzerland,Denmark,Norway,Canada,Finland,Netherlands,Sweden,New Zealand,Australia,Israel,Austria,Mexico,United States,Ireland,United Kingdom,Germany,Year
Happiness Score 2015,7.587,7.527,7.522,7.427,7.406,7.378,7.364,7.286,7.284,7.278,7.200,7.187,7.119,NaN,NaN,NaN,2015
Happiness Score 2016,7.509,7.526,7.498,7.404,7.413,7.339,7.291,7.334,7.313,7.267,7.119,NaN,7.104,NaN,NaN,NaN,2016
Happiness Score 2017,7.494,7.522,7.537,7.316,7.469,7.377,7.284,7.314,7.284,7.213,7.006,NaN,6.993,6.977,NaN,NaN,2017
Happiness Score 2018,7.487,7.555,7.594,7.328,7.632,7.441,7.314,7.324,7.272,NaN,7.139,NaN,NaN,6.977,7.190,6.965,2018
Happiness Score 2019,7.480,7.600,7.554,7.278,7.769,7.488,7.343,7.307,7.228,7.139,7.246,NaN,NaN,NaN,7.054,NaN,2019


In [29]:
wh_2015_2019_df = wh_2015_2019_df.transpose()
wh_2015_2019_df


,Happiness Score 2015,Happiness Score 2016,Happiness Score 2017,Happiness Score 2018,Happiness Score 2019
Country,,,,,
Switzerland,7.587,7.509,7.494,7.487,7.480
Denmark,7.527,7.526,7.522,7.555,7.600
Norway,7.522,7.498,7.537,7.594,7.554
Canada,7.427,7.404,7.316,7.328,7.278
Finland,7.406,7.413,7.469,7.632,7.769
Netherlands,7.378,7.339,7.377,7.441,7.488
Sweden,7.364,7.291,7.284,7.314,7.343
New Zealand,7.286,7.334,7.314,7.324,7.307
Australia,7.284,7.313,7.284,7.272,7.228


In [39]:
wh_2015_2019_df = wh_2015_2019_df.drop(index="Year")

total_scores = wh_2015_2019_df.sum()
total_scores

Happiness Score 2015    95.565
Happiness Score 2016    88.117
Happiness Score 2017    94.786
Happiness Score 2018    95.218
Happiness Score 2019    88.486
dtype: float64

In [67]:
# Map etfs to countries

etf_country_map = {
    "Finland":"iShares MSCI Finland Capped",
    "Denmark":"iShares MSCI Denmark Capped",
    "Norway":"iShares MSCI Norway Capped",
    "Netherlands":"iShares MSCI Netherlands",
    "Switzerland":"iShares MSCI Switzerland Capped",
    "Sweden":"iShares MSCI Sweden Capped",
    "New Zealand":"iShares MSCI New Zealand Capped",
    "Canada":"iShares MSCI Canada",
    "Austria":"iShares MSCI Austria Capped",
    "Australia":"iShares MSCI Australia",
    "Israel":"iShares MSCI Israel Capped",
    "United Kingdom":"iShares MSCI United Kingdom",
    "United States":"ishares S&P 500",
    "Ireland":"iShares MSCI Ireland",
    "Germany":"iShares Currency Hedged MSCI Germany",
    "Mexico":"iShares MSCI Mexico Capped",
    "Benchmark":"iShares MSCI World"
}

In [73]:
etf_country_map_df = pd.DataFrame.from_dict(etf_country_map, orient='index')
etf_country_map_df.columns = ["ETF"]
etf_country_map_df.index.name = "Country"
etf_country_map_df

,ETF
Country,
Finland,iShares MSCI Finland Capped
Denmark,iShares MSCI Denmark Capped
Norway,iShares MSCI Norway Capped
Netherlands,iShares MSCI Netherlands
Switzerland,iShares MSCI Switzerland Capped
Sweden,iShares MSCI Sweden Capped
New Zealand,iShares MSCI New Zealand Capped
Canada,iShares MSCI Canada
Austria,iShares MSCI Austria Capped


In [116]:
country_etf_combined = pd.merge(etf_country_map_df, wh_2015_2019_df,on='Country', how='outer')

country_etf_combined.set_index('ETF', inplace=True)
columns = [2015,2016,2017,2018,2019]
country_etf_combined.columns = columns

country_etf_combined


,2015,2016,2017,2018,2019
ETF,,,,,
iShares MSCI Finland Capped,7.406,7.413,7.469,7.632,7.769
iShares MSCI Denmark Capped,7.527,7.526,7.522,7.555,7.600
iShares MSCI Norway Capped,7.522,7.498,7.537,7.594,7.554
iShares MSCI Netherlands,7.378,7.339,7.377,7.441,7.488
iShares MSCI Switzerland Capped,7.587,7.509,7.494,7.487,7.480
iShares MSCI Sweden Capped,7.364,7.291,7.284,7.314,7.343
iShares MSCI New Zealand Capped,7.286,7.334,7.314,7.324,7.307
iShares MSCI Canada,7.427,7.404,7.316,7.328,7.278
iShares MSCI Austria Capped,7.200,7.119,7.006,7.139,7.246


In [122]:
country_etf_weighted = country_etf_combined

country_etf_weighted["2015 Weights"] = (country_etf_weighted[2015]/country_etf_weighted[2015].sum()) * 100
country_etf_weighted["2016 Weights"] = (country_etf_weighted[2016]/country_etf_weighted[2016].sum()) * 100
country_etf_weighted["2017 Weights"] = (country_etf_weighted[2017]/country_etf_weighted[2017].sum()) * 100
country_etf_weighted["2018 Weights"] = (country_etf_weighted[2018]/country_etf_weighted[2018].sum()) * 100
country_etf_weighted["2019 Weights"] = (country_etf_weighted[2019]/country_etf_weighted[2019].sum()) * 100

country_etf_weighted = country_etf_weighted.drop(columns=[2015,2016,2017,2018,2019])

columns = [2015,2016,2017,2018,2019]
country_etf_weighted.columns = columns

country_etf_weighted


,2015,2016,2017,2018,2019
ETF,,,,,
iShares MSCI Finland Capped,7.749699,8.412679,7.879856,8.015291,8.779920
iShares MSCI Denmark Capped,7.876315,8.540917,7.935771,7.934424,8.588929
iShares MSCI Norway Capped,7.871083,8.509141,7.951596,7.975383,8.536944
iShares MSCI Netherlands,7.720400,8.328699,7.782795,7.814699,8.462356
iShares MSCI Switzerland Capped,7.939099,8.521625,7.906231,7.863009,8.453315
iShares MSCI Sweden Capped,7.705750,8.274226,7.684679,7.681321,8.298488
iShares MSCI New Zealand Capped,7.624130,8.323025,7.716330,7.691823,8.257803
iShares MSCI Canada,7.771674,8.402465,7.718439,7.696024,8.225030
iShares MSCI Austria Capped,7.534139,8.079031,7.391387,7.497532,8.188866


In [119]:
country_etf_weighted.sum()

2015 Weights    100.0
2016 Weights    100.0
2017 Weights    100.0
2018 Weights    100.0
2019 Weights    100.0
dtype: float64

In [120]:
# Annual Return
all_etf_df_return_2015 = all_etf_df.loc['2015-01-01':'2015-12-31'].mean() * 12
all_etf_df_return_2015 = pd.DataFrame(all_etf_df_return_2015, columns=[2015])

all_etf_df_return_2016 = all_etf_df.loc['2016-01-01':'2016-12-31'].mean() * 12
all_etf_df_return_2016 = pd.DataFrame(all_etf_df_return_2016, columns=[2016])

all_etf_df_return_2017 = all_etf_df.loc['2017-01-01':'2017-12-31'].mean() * 12
all_etf_df_return_2017 = pd.DataFrame(all_etf_df_return_2017, columns=[2017])

all_etf_df_return_2018 = all_etf_df.loc['2018-01-01':'2018-12-31'].mean() * 12
all_etf_df_return_2018 = pd.DataFrame(all_etf_df_return_2018, columns=[2018])

all_etf_df_return_2019 = all_etf_df.loc['2019-01-01':'2019-12-31'].mean() * 12
all_etf_df_return_2019 = pd.DataFrame(all_etf_df_return_2019, columns=[2019])

etf_annual_returns = pd.concat([all_etf_df_return_2015, all_etf_df_return_2016, all_etf_df_return_2017, all_etf_df_return_2018, all_etf_df_return_2019], axis=1)

etf_annual_returns

,2015,2016,2017,2018,2019
iShares MSCI Finland Capped,0.002996,0.019479,0.186938,-0.092812,0.076145
iShares MSCI Denmark Capped,0.189833,-0.080276,0.290187,-0.163079,0.199443
iShares MSCI Norway Capped,-0.198335,0.149565,0.181443,-0.098098,0.097274
iShares MSCI Netherlands,0.005124,0.022020,0.279715,-0.175988,0.262903
iShares MSCI Switzerland Capped,-0.013449,-0.045295,0.192891,-0.113169,0.259286
iShares MSCI Sweden Capped,-0.105530,-0.004391,0.171200,-0.168124,0.168422
iShares MSCI New Zealand Capped,-0.007624,0.079026,0.186421,-0.023088,0.235747
iShares MSCI Canada,-0.210315,0.205191,0.128674,-0.200510,0.232654
iShares MSCI Austria Capped,0.100030,0.074361,0.411082,-0.274014,0.137626
iShares MSCI Australia,-0.141334,0.079060,0.140252,-0.177539,0.166867


In [123]:
weighted_return = etf_annual_returns * country_etf_weighted
weighted_return

,2015,2016,2017,2018,2019
iShares MSCI Finland Capped,0.023218,0.163867,1.473041,-0.743916,0.668544
iShares MSCI Denmark Capped,1.495183,-0.685632,2.302855,-1.293936,1.713000
iShares MSCI Norway Capped,-1.561109,1.272666,1.442762,-0.782371,0.830426
iShares MSCI Netherlands,0.039557,0.183400,2.176968,-1.375289,2.224775
iShares MSCI Switzerland Capped,-0.106776,-0.385987,1.525041,-0.889847,2.191825
iShares MSCI Sweden Capped,-0.813184,-0.036330,1.315616,-1.291412,1.397645
iShares MSCI New Zealand Capped,-0.058129,0.657734,1.438490,-0.177588,1.946751
iShares MSCI Canada,-1.634497,1.724114,0.993165,-1.543131,1.913584
iShares MSCI Austria Capped,0.753639,0.600766,3.038464,-2.054432,1.127002
iShares MSCI Australia,-1.077252,0.656135,1.077790,-1.355902,1.363055


In [83]:
# pull this in from countries pkl file when available

# country_score_2019 = {
#     "Finland":7.769, 
#     "Denmark":7.6,
#     "Norway":7.554,
#     "Netherlands":7.494,
#     "Switzerland":7.488,
#     "Sweden":7.48,
#     "New Zealand":7.307, 
#     "Canada":7.278, 
#     "Austria":7.246, 
#     "Australia":7.228, 
#     "Israel":7.167, 
#     "United Kingdom":7.054
# }


In [43]:
# sum_2019_scores = sum(country_score_2019.values())


In [44]:
# not used in any calc but may be useful to show countries that we couldn't get data for / change in number of countried included per year

number_countries_2019 = len(country_score_2019)


# Portfolio Weightings

In [88]:
# nineteen_df = pd.DataFrame(wh_2015_2019_df['Happiness Score 2019'])

# nineteen_df


In [82]:
# portfolio_weight_2019 = {}

# for country in nineteen_df:
#     etf = etf_country_map[country] 
#     weighting = nineteen_df[country]/sum_2019_scores
#     portfolio_weight_2019.update({etf:weighting})

# portfolio_weight_2019

## Monthly Portfolio Return vs Benchmark

In [ ]:
# Empty dataframe to collect portfolio weights from below for loop
portfolio_2019_df = pd.DataFrame()

# Return * Portfolio Weight
for etf in portfolio_weight_2019:
    etf_column = all_etf_df[etf] * portfolio_weight_2019[etf]
    portfolio_2019_df[etf] = etf_column

# Filter Dates
portfolio_2019_df = portfolio_2019_df.loc['2019-01-01':'2019-12-31']
    
# Combined return for all funds and rename Happy Portfolio
portfolio_2019_df = pd.DataFrame(portfolio_2019_df.sum(axis=1), columns=['Happy Portfolio'])

# Add Benchmark
benchmark = "iShares MSCI World"

benchmark_return = pd.DataFrame(all_etf_df[benchmark])

# slice by date
benchmark_return = benchmark_return.loc["2019-01-01":"2019-12-31"]

portfolio_2019_df = pd.concat([portfolio_2019_df,benchmark_return], axis=1)
portfolio_2019_df

## Cumulative Portfolio Return vs Benchmark

In [ ]:
# cumulative returns
cum_portfolio_2019 = (1 + portfolio_2019_df).cumprod()
cum_portfolio_2019


In [ ]:
# Plot 2019 vs benchmark

fig = px.line(
    cum_portfolio_2019,
    title="Cumulative Return of Happy Portfolio vs Benchmark 2019",
    height=500,
    width=1000,
    labels={
        "value":"Return",
        "date":"Date",
        "variable":""
    }
)
fig.show()

In [ ]:
## Portfolio and Benchmark Statistics

In [ ]:
portfolio_2019_df.describe()

In [ ]:
fig = px.box(
    portfolio_2019_df,
    title='Box Plot of Happy Portfolio and Benchmark Returns'
)
fig.show()

In [ ]:
# Sharpe Ratio

average_annual_return = portfolio_2019_df.mean() * 12
display(average_annual_return)

annual_sd_portfolio = portfolio_2019_df.std() * np.sqrt(12)
display(annual_sd_portfolio)

sharpe_ratios = average_annual_return / annual_sd_portfolio
display(sharpe_ratios)

fig = px.bar(
    sharpe_ratios,
    title="Sharpe Ratios of Happy Fund and Benchmark"
)
fig.show()

In [ ]:
# PULL IN WORLD HAPPINESS DF TO OBTAIN RANKINGS OR SCORE
# CREATE WEIGHTINGS VARIABLES PER YEAR
# CREATE PERFORMANCE VARIABLES BY YEAR BASED ON WEIGHTINGS AND RANKINGS 

